In [ ]:
# import module

!pip install selenium
from bs4 import BeautifulSoup
from selenium import webdriver
import time

In [ ]:
# main

column_name = ['사이트', '아우터1', '아우터2', '아우터3', '아우터4', '상의1', '상의2','상의3', '상의4', '바지1', '바지2', '신발', '악세사리1', '악세사리2', '악세사리3','악세사리4']
each_cody_df = []
column_name_detail = ['사이트', 'hashtag', '아우터1', '아우터2', '아우터3', '아우터4', '상의1', '상의2','상의3', '상의4', '바지1', '바지2', '신발', '악세사리1', '악세사리2', '악세사리3','악세사리4']
each_cody_detail_df = []
each_item_detail_df = []

url = 'https://www.musinsa.com/app/styles/lists?use_yn_360=&style_type=&brand=&model=&tag_no=&max_rt=&min_rt=&display_cnt=60&list_kind=big&sort=date&page='
start_page = 1
page_target = 423 #424페이지 까지 있다 
driver = webdriver.Chrome('./chromedriver.exe')

item_infos = []

for q in range(start_page, page_target):
    # url 에 q를 더해줘서 taeget page까지의 url 을 가져온다
    temp_url = url + str(q + 1)
    driver.get(temp_url)
    if q == 0:
        driver.maximize_window()
    time.sleep(2)

    # 각 코디에 접속
    for i in range(1, 61):
        each_cody = [0] * len(column_name)
        each_cody_detail = [0] * (len(column_name) + 1)
        each_item_detail = []
        driver.find_element(By.XPATH, '/html/body/div[3]/div[3]/form/div[4]/div/ul/li[{}]'.format(i)).click()
        time.sleep(2)
        page = driver.page_source
        soup = BeautifulSoup(page, "html.parser")
        contents = []
        tmp = soup.find('div', 'ui-tag-list').find_all('a')
        for i in tmp:
            contents.append(str(i.text).replace('#', ''))

        color_eng = ['red', 'orange', 'yellow', 'green', 'blue', 'pupple', 'pink', 'beige', 'mint', 'brown', 'white',
                     'black', 'silver', 'cream', 'grey', 'navy']
        color_kor = ['레드', '오렌지', '옐로우', '그린', '블루', '퍼플', '핑크', '베이지', '민트', '브라운', '화이트', '블랙', '실버', '크림', '그레이',
                     '네이비']
        color_ENG = ['RED', 'ORANGE', 'YELLOW', 'GREEN', 'BLUE', 'PUPPLE', 'PINK', 'BEIGE', 'MINT', 'BROWN', 'WHITE',
                     'BLACK', 'SILVER', 'CREAM', 'GREY', 'NAVY']
        special_word = "!#$%^&*()_"
        current_url = driver.current_url

        each_cody[0] = current_url
        each_cody_detail[0] = current_url
        each_cody_detail[1] = contents
        
        # 각 제품별 링크 저장
        temp_link_list = []
        for k in soup.find_all('a', 'styling_img'):
            temp_link_list.append('https://www.musinsa.com/' + k['href'])
        
        # 각 제품별 링크에 접속 / 크롤링
        for i in temp_link_list:
            temp_list = []
            driver.get(i)

            page = driver.page_source
            soup = BeautifulSoup(page, "html.parser")
            category = soup.find('div', 'product_info').find('p').find_all('a')
            for i in category:
                text = i.text.strip()
                if text != '스포츠/용품':
                    if text == '스포츠모자':
                        text = '모자'
                    elif text == '스포츠가방':
                        text = '가방'
                    elif text == '스포츠신발':
                        text = '신발'
                    temp_list.append(text)
                    break

            # 특문 제거
            temp_title = soup.find('span', 'product_title').text
            temp_title = temp_title.replace('\n', ' ')
            for k in special_word:
                temp_title = temp_title.replace(k, '')
            temp_list.append(temp_title)

            # 색상 추가
            temp_color = []
            for j in temp_title.split(' '):
                if j in color_eng:
                    temp_color.append(j)
                elif j in color_ENG:
                    temp_color.append(color_eng[color_ENG.index(j)])
                elif j in color_kor:
                    temp_color.append(color_eng[color_kor.index(j)])
                else:
                    temp_color.append('not shown')
            temp_color = list(set(temp_color))
            temp_list.append(temp_color[0])

            # 링크 추가
            temp_link = driver.current_url
            temp_list.append(temp_link)

            # temp_list 추가
            item_infos.append(temp_list)
            # print(temp_list)
            each_item_detail.append(temp_list)

        driver.get(temp_url)
        
        # 결과 저장
        outer = 1
        top = 5
        pants = 9
        shoes = 11
        accessory = 12
        try:
            for i in each_item_detail:
                if i[0] == '아우터':
                    each_cody[outer] = i[2]
                    each_cody_detail[outer + 1] = i[3]
                    outer += 1
                elif i[0] == '상의':
                    each_cody[top] = i[2]
                    each_cody_detail[top + 1] = i[3]
                    top += 1
                elif i[0] == '바지' or i[0] == '하의':
                    each_cody[pants] = i[2]
                    each_cody_detail[pants + 1] = i[3]
                    pants += 1
                elif i[0] == '신발' or i[0] == '스니커즈':
                    each_cody[shoes] = i[2]
                    each_cody_detail[shoes + 1] = i[3]
                else:
                    if accessory < 16:
                        each_cody[accessory] = i[2]
                        each_cody_detail[accessory + 1] = i[3]
                        accessory += 1
        except:
            print('ERROR')
        '''       
        print("EACH CODY: ")
        print(each_cody)
        print('----------------------------------------------')
        print("EACH CODY DETAIL: ")
        print(each_cody_detail)
        print('----------------------------------------------')
        print(each_item_detail)
        print('----------------------------------------------')
        '''

        each_cody_df.append(each_cody)
        each_cody_detail_df.append(each_cody_detail)
        each_item_detail_df.append(each_item_detail)

In [ ]:
# file maker

columns1 = ['주소', '아우터1', '아우터2', '아우터3', '아우터4', '상의1', '상의2', '상의3', '상의4', '하의1', '하의2', 
           '신발', '악세사리1', '악세사리2', '악세사리3', '악세사리4']

columns2 = ['주소', '키워드', '아우터1', '아우터2', '아우터3', '아우터4', '상의1', '상의2', '상의3', '상의4', '하의1', '하의2', 
           '신발', '악세사리1', '악세사리2', '악세사리3', '악세사리4']

columns3 = ['계열', '제품명', '색상', '주소']

with open('filename_cody_style.csv', 'w') as f:
    column = ','.join(columns1) + '\n'
    f.write(column)
    f.close()
with open('filename_cody_detail.csv', 'w') as f:
    column = ','.join(columns2) + '\n'
    f.write(column)
    f.close()
with open('filename_item_detail.csv', 'w') as f:
    column = ','.join(columns3) + '\n'
    f.write(column)
    f.close()

In [ ]:
# cody_style # color

with open('filename_cody_style.csv', 'a') as f:
    for i in each_cody_df:
        raw = ','.join(map(str, i)) + "\n"
        f.write(raw)
    f.close()

In [ ]:
# cody detail # link

with open('filename_cody_detail.csv', 'a') as f:
    for i in each_cody_detail_df:
        raw1 = [i[0]]
        raw2 = ''
        for j in i[1]:
            raw2 = raw2 + j + ' '
        raw2 = [raw2]
        raw3 = i[2:]
        total_raw = raw1 + raw2 + raw3
        raw = ','.join(map(str, total_raw)) + "\n"
        f.write(raw)
    f.close()

In [ ]:
# item detail

with open('filename_item_detail.csv', 'a') as f: # 한글 깨질 시 encoding = 'utf-8' or 'utf-8-sig'
    for i in each_item_detail_df:
        for j in i:
            raw1 = [j[0]]
            raw2 = [j[1].replace(',', '')]
            raw3 = j[2:]
            total_raw = raw1 + raw2 + raw3
            raw = ','.join(map(str, total_raw)) + "\n"
            f.write(raw)
    f.close()